In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Loading Data

In [ ]:





import json
import pandas as pd
!pip install alive-progress
from alive_progress.styles import showtime
from alive_progress import alive_bar
import time
from time import sleep
from tqdm.notebook import tqdm




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 4.9 MB/s eta 0:00:00
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210082 sha256=d3b226e281986527b406941628adf9b45d2edc074a1d01868f2020988550eef6
  Stored in directory: /root/.cache/pip/wheels/01/e1/49/37e6bde9886439057450c494a79b0bef8bbe897a54aebfc757
Successfully built grapheme


In [ ]:
#loading scores
with open(r'/content/drive/MyDrive/Level Data Datasets/2023 District 18 Scores.json') as scores_data:
    sc_json = json.load(scores_data)

    #assign dataframe
    ld_scores = pd.DataFrame(sc_json['SELECT s.id, s.districtId, sc.*\r\nFROM aim2.students s\r\nJOIN aim2.scores sc ON s.id = sc.studentId \r\nWHERE s.districtId = 18'])
    # we will be making many changes to ld_scores, let us save for later the original dataframe.
    # note: this cell takes a LONG time


In [ ]:
    # we will be making many changes to ld_scores, let us save for later the original dataframe.
    # note: this cell takes a LONG time
    ld_scores_original = ld_scores.copy()

In [ ]:
#loading benchmarks
with open(r'/content/drive/MyDrive/Level Data Datasets/benchmarks_202410011642.json') as benchmarks_data:
    benchmarks_json = json.load(benchmarks_data)
    ld_benchmarks = pd.DataFrame(benchmarks_json['benchmarks'])

In [ ]:
#load schools
with open(r'/content/drive/MyDrive/Level Data Datasets/District 18 Anonymized Schools.json') as schools_data:
    schools_json = json.load(schools_data)
    #assign dataframe
    ld_schools = pd.DataFrame(schools_json['schools'])

In [ ]:
#load vendor student usage
with open(r'/content/drive/MyDrive/Level Data Datasets/2023 District 18 Vendor Student Usage.json') as vendor_data:
    vendor_json = json.load(vendor_data)
    #assign dataframe
    ld_vendorUsage = pd.DataFrame(vendor_json['SELECT s.districtId, sc.studentId, sc.`year` , sc.vendorId, sc.active, sc.usageTypeId, sc.weeklyUsageMinutes \r\nFROM aim2.students s\r\nJOIN aim2.vendorStudentUsage sc ON s.id = sc.studentId \r\nWHERE s.districtId = 18 AND sc.`year` = 2023'])

In [ ]:
#load student attributes
with open(r'/content/drive/MyDrive/Level Data Datasets/2023 District 18 Student Attributes.json') as student_attributes_data:
    student_attributes_json = json.load(student_attributes_data)
    ld_attributes = pd.DataFrame(student_attributes_json['SELECT s.districtId, sc.studentId, sc.`year` , sc.value \r\nFROM aim2.students s\r\nJOIN aim2.studentCustomAttributes sc ON s.id = sc.studentId \r\nWHERE s.districtId = 18'])

In [ ]:
#load vendor usage types
with open(r'/content/drive/MyDrive/Level Data Datasets/District 18 Vendor Usage Types.json') as vendor_ref_data:
    vendor_ref_json = json.load(vendor_ref_data)
    ld_vendorTypes = pd.DataFrame(vendor_ref_json['vendorUsageTypes'])

# Vendor Inspection
Finding exactly what columns we need to include in our scores datatable

In [ ]:
ld_vendorTypes['name'].unique()

array(['IXL Reading Non', 'IXL Reading Partial', 'IXL Reading User',
       'IXL Math Non User', 'IXL Math Partial ', 'IXL Math User',
       'MyOn R User', 'MyOn R Partial User', 'MyOn R Non User',
       'Freckle R User', 'Freckle R Partial User', 'Freckle R Non User',
       'Freckle M User', 'Freckle M Partial User', 'Freckle M Non User ',
       'Freckle Sci User', 'Freckle Sci Partial User',
       'Freckle Sci NonUser', 'Freckle SS User', 'Freckle SS PartialUser',
       'Freckle SS NonUser', 'IXL Science User', 'IXL Science Non User',
       'IXL Science Partial', 'IXL SS User', 'IXL SS Non',
       'IXL SS Partial', 'Freckle M Adaptive User',
       'Freckle M Adaptive Partial User', 'Freckle M Adaptive Non User',
       'Freckle M Target User', 'Freckle M Target Partial User',
       'Freckle M Target Non User', 'Beable ELA User',
       'Beable ELA Partial User', 'Beable Non User', 'AR User',
       'AR Partial User', 'AR Non User', 'Reflex M User',
       'Reflex M Partial 

In [ ]:
#'Beable Non User' doesn't match the 'Beable ELA [usetype]' syntax, change to 'Beable ELA Non User'
ld_vendorTypes['name'] = ld_vendorTypes['name'].replace('Beable Non User', 'Beable ELA Non User')

In [ ]:
ld_vendorTypes['usageTypeId'].value_counts()
#we need to cut down on this. Let us inspect to make certain that these repeat values are useless

,count
usageTypeId,
2,20
1,19
3,19
7,15
8,13
9,13
4,10
5,10
6,10


In [ ]:
ld_attributes.value.value_counts()

,count
value,
11,203
12,134


In [ ]:
def auditNameSymmetry(usageTypeId, df): #may expand this function to check any dataframe for symmetry in any column. for now, unneccessary
  df_subset = df[df['usageTypeId']==usageTypeId]['name']
  if (df_subset.value_counts().shape[0] != 1):
    return False
  return True

  #this function ensures that for a given dataframe and usagetypeid, the name is the same for all elements


In [ ]:
for usageTypeId in (ld_vendorTypes['usageTypeId'].unique()):
  print(auditNameSymmetry(usageTypeId, ld_vendorTypes)) #great, we know that extra values are redundant. By inspection, they only differ by initiativeId

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


Alright, let us now trim the fat on the vendorTypes. Let us make a new dataframe.

In [ ]:
ld_trimmedVendorTypes = ld_vendorTypes[['usageTypeId', 'name','utilBucketType','id']]
ld_trimmedVendorTypes.head()

,usageTypeId,name,utilBucketType,id
0,1,IXL Reading Non,nonUser,4
1,2,IXL Reading Partial,partialUser,5
2,3,IXL Reading User,fullUser,6
3,4,IXL Math Non User,nonUser,7
4,5,IXL Math Partial,partialUser,8


In [ ]:
ld_trimmedVendorTypes = ld_trimmedVendorTypes.drop_duplicates(subset=['usageTypeId'], keep='first')
ld_trimmedVendorTypes = ld_trimmedVendorTypes.reset_index(drop=True)
ld_trimmedVendorTypes['usageTypeId'].value_counts()

,count
usageTypeId,
1,1
23,1
27,1
26,1
28,1
29,1
30,1
31,1
32,1


Great. Now we have to find out all the columns we need to add. Really, we need to find all the product names. Let Gemini do this, and verify results.

In [ ]:

import re

def extract_product_name(name):
  """
  Extracts the product name from a vendor usage type name by removing
  keywords like 'Partial', 'Non User', 'NonUser', or 'User'.
  """
  name = re.sub(r'\b(Partial|Non User|NonUser|User|Non|PartialUser)\b', '', name)
  name = name.strip()
  return name

# Apply the function to the 'name' column and create a new column 'product_name'
ld_trimmedVendorTypes['product_name'] = ld_trimmedVendorTypes['name'].apply(extract_product_name)

# Print the updated DataFrame to verify
print(ld_trimmedVendorTypes[['name', 'product_name']])


                               name        product_name
0                   IXL Reading Non         IXL Reading
1               IXL Reading Partial         IXL Reading
2                  IXL Reading User         IXL Reading
3                 IXL Math Non User            IXL Math
4                 IXL Math Partial             IXL Math
5                     IXL Math User            IXL Math
6                       MyOn R User              MyOn R
7               MyOn R Partial User              MyOn R
8                   MyOn R Non User              MyOn R
9                    Freckle R User           Freckle R
10           Freckle R Partial User           Freckle R
11               Freckle R Non User           Freckle R
12                   Freckle M User           Freckle M
13           Freckle M Partial User           Freckle M
14              Freckle M Non User            Freckle M
15                 Freckle Sci User         Freckle Sci
16         Freckle Sci Partial User         Frec

This looks great. Now, let's make a column to associate usage type with a number 0-2. Let Gemini do this too.

In [ ]:
ld_trimmedVendorTypes

,usageTypeId,name,utilBucketType,id,product_name
0,1,IXL Reading Non,nonUser,4,IXL Reading
1,2,IXL Reading Partial,partialUser,5,IXL Reading
2,3,IXL Reading User,fullUser,6,IXL Reading
3,4,IXL Math Non User,nonUser,7,IXL Math
4,5,IXL Math Partial,partialUser,8,IXL Math
5,6,IXL Math User,fullUser,9,IXL Math
6,7,MyOn R User,fullUser,853,MyOn R
7,8,MyOn R Partial User,partialUser,854,MyOn R
8,9,MyOn R Non User,nonUser,855,MyOn R
9,10,Freckle R User,fullUser,1130,Freckle R


In [ ]:
def map_util_bucket_type(util_bucket_type):
  """Maps utilBucketType to an integer."""
  if util_bucket_type == 'nonUser':
    return 0
  elif util_bucket_type == 'partialUser':
    return 1
  elif util_bucket_type == 'fullUser':
    return 2
  else:
    return -1

# Apply the function to create the new column
ld_trimmedVendorTypes['productUse_int'] = ld_trimmedVendorTypes['utilBucketType'].apply(map_util_bucket_type)

# Print the updated DataFrame to verify
#print(ld_trimmedVendorTypes[['utilBucketType', 'productUse_int']])
# Looks good, let's not clutter

In [ ]:
ld_trimmedVendorTypes

,usageTypeId,name,utilBucketType,id,product_name,productUse_int
0,1,IXL Reading Non,nonUser,4,IXL Reading,0
1,2,IXL Reading Partial,partialUser,5,IXL Reading,1
2,3,IXL Reading User,fullUser,6,IXL Reading,2
3,4,IXL Math Non User,nonUser,7,IXL Math,0
4,5,IXL Math Partial,partialUser,8,IXL Math,1
5,6,IXL Math User,fullUser,9,IXL Math,2
6,7,MyOn R User,fullUser,853,MyOn R,2
7,8,MyOn R Partial User,partialUser,854,MyOn R,1
8,9,MyOn R Non User,nonUser,855,MyOn R,0
9,10,Freckle R User,fullUser,1130,Freckle R,2


In [ ]:
venProductList = ld_trimmedVendorTypes['product_name'].unique()
venProductList
# venProductList = ['IXL Reading', 'IXL Math', 'MyOn R', 'Freckle R', 'Freckle M', 'Freckle Sci', 'Freckle SS', 'IXL Science', 'IXL SS', 'Freckle M Adaptive', 'Freckle M Target', 'Beable ELA', 'AR User', 'Reflex M', 'Frax M']

array(['IXL Reading', 'IXL Math', 'MyOn R', 'Freckle R', 'Freckle M',
       'Freckle Sci', 'Freckle SS', 'IXL Science', 'IXL SS',
       'Freckle M Adaptive', 'Freckle M Target', 'Beable ELA', 'AR',
       'Reflex M', 'Frax M'], dtype=object)

We want to add a number of columns to our ld_scores dataframe. First, we want to add and populate all the product use columns. Second, we want to one hot encode the attributes that we have data for. The first one is harder and more important, let's start there.

In [ ]:
ld_vendorUsage.head()

,districtId,studentId,year,vendorId,active,usageTypeId,weeklyUsageMinutes
0,18,1480117,2023,140472,0,39,4
1,18,1480117,2023,11333,0,6,0
2,18,1480117,2023,72609,0,3,0
3,18,1480118,2023,140472,0,39,4
4,18,1480118,2023,11333,0,6,0
...,...,...,...,...,...,...,...
70310,18,1804199,2023,11333,0,4,0
70311,18,1804199,2023,72609,0,1,0
70312,18,1804201,2023,11333,0,4,0
70313,18,1804201,2023,72609,0,1,0


We need to associate each item here with the scores data set by studentID. Then, we need to light up the rows product with its productUse_int from trimmed vendor types. First, let us trim vendorUsage for this purpose, and append the necessary columns to the scores dataset.

TODO for later, should we start caring about weekly usage minutes?

In [ ]:
ld_trimmedVendorUsage = ld_vendorUsage[['studentId', 'usageTypeId']]
ld_trimmedVendorUsage.head()

,studentId,usageTypeId
0,1480117,39
1,1480117,6
2,1480117,3
3,1480118,39
4,1480118,6


In [ ]:
for product_name in venProductList:
  ld_scores["user_" + product_name] = 0

Before we make a bunch of changes to ld_scores, why don't we take out all the columns we don't need? We will also sort by studentId so any search algorithms we do can use binary search

In [ ]:
# TODO, take out test scores that are useless
ld_scores_original['studentId'].value_counts()

,count
studentId,
1483818,19
1802905,19
1490116,19
1487272,19
1803268,19
...,...
1492164,1
1492170,1
1804160,1


In [ ]:
len(ld_vendorUsage['studentId'].unique())

11654

In [ ]:
ld_vendorUsage[ld_vendorUsage['studentId']==1488823]

,districtId,studentId,year,vendorId,active,usageTypeId,weeklyUsageMinutes
42138,18,1488823,2023,11333,0,6,0
42139,18,1488823,2023,72609,0,2,0
42140,18,1488823,2023,140472,0,37,120
42141,18,1488823,2023,140574,0,40,0


In [ ]:
ld_scores[ld_scores['studentId']==1488823]

,id,districtId,subgroup_TCAPELALevel,ticket,studentId,studentLevel,year,scoreDate,measurementTypeId,subgroup_specialEd,...,user_Freckle Sci,user_Freckle SS,user_IXL Science,user_IXL SS,user_Freckle M Adaptive,user_Freckle M Target,user_Beable ELA,user_AR,user_Reflex M,user_Frax M
88830,4707976,18,0,,1488823,3,2023,0000-00-00,9,3,...,0,0,0,0,0,0,0,0,0,0
88831,4713836,18,0,,1488823,3,2023,0000-00-00,4,3,...,0,0,0,0,0,0,0,0,0,0
88832,5426110,18,0,,1488823,3,2023,0000-00-00,5,3,...,0,0,0,0,0,0,0,0,0,0
88833,7004194,18,0,,1488823,3,2023,0000-00-00,6,3,...,0,0,0,0,0,0,0,0,0,0
88834,7692582,18,0,,1488823,3,2023,0000-00-00,1,3,...,0,0,0,0,0,0,0,0,0,0
88835,7692583,18,0,,1488823,3,2023,0000-00-00,2,3,...,0,0,0,0,0,0,0,0,0,0
88836,7692584,18,0,,1488823,3,2023,0000-00-00,3,3,...,0,0,0,0,0,0,0,0,0,0
88837,7692585,18,0,,1488823,3,2023,0000-00-00,7,3,...,0,0,0,0,0,0,0,0,0,0
88838,7692586,18,0,,1488823,3,2023,0000-00-00,8,3,...,0,0,0,0,0,0,0,0,0,0
88839,7692587,18,0,,1488823,3,2023,0000-00-00,10,3,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
len(ld_scores['studentId'].unique())

11633

Let us now populate the scores data with our data from vendorUsage. However, we will check to see if they conflict for any product. We will construct an ld_scores prime dataframe to hold each unique studentID, which we will then populate with our 'user_*' values

In [ ]:
to_drop = ['ticket', 'subgroup_TCAPELALevel', 'scoreDate', 'subgroup_ethnicity']
ld_scores = ld_scores.drop(columns=to_drop)
#ld_scores.head()

In [ ]:
ld_scores = ld_scores.sort_values('studentId')


In [ ]:
ld_scores.head()

,id,districtId,studentId,studentLevel,year,measurementTypeId,subgroup_specialEd,subgroup_lunchStatus,subgroup_gender,subgroup_ell,...,user_Freckle Sci,user_Freckle SS,user_IXL Science,user_IXL SS,user_Freckle M Adaptive,user_Freckle M Target,user_Beable ELA,user_AR,user_Reflex M,user_Frax M
0,3268686,18,1480117,11,2023,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4933778,18,1480117,11,2023,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5441518,18,1480117,11,2023,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7263146,18,1480117,11,2023,6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7510656,18,1480117,11,2023,7,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Consolidating student data
Before we populate this user data, it will save us time to consolidate the test scores from all these students. Let us average all the the test scores as an approximation of student proficiency.

In [ ]:
def find_non_specified_uniques_from_column(df, column_name, values_to_remove):
    """Removes specified values from a column in a DataFrame and returns unique values.
    Returns:
      A list of unique values in the specified column after removing the specified values.
      Returns an empty list if the column does not exist or is empty.
      This will be used to return every non-zero value
    """
    if column_name not in df.columns:
        print(f"Error: Column '{column_name}' not found in DataFrame.")
        return []

    unique_values = df[column_name].unique().tolist()
    for value in values_to_remove:
      if value in unique_values:
        unique_values.remove(value)
    cleaned_unique_values = unique_values
    return cleaned_unique_values


In [ ]:
def calculate_averages_for_student(student_data):
    """Calculates averages for columns with multiple values for a single student.

    Args:
        student_data (pd.DataFrame): DataFrame containing data for a single student.

    Returns:
        pd.Series: A Series containing the student ID and the averages for relevant columns.
                      Returns None if no columns have multiple unique values other than 0
    """
    if(student_data.empty):
      return None
    if(student_data.shape[0]==1):
      return student_data
    redundant_rows = range(1, (student_data.shape[0]),1)

    result_df = student_data.copy().reset_index(drop=True).drop(index=redundant_rows)
    student_data = student_data.drop(['id','measurementTypeId'], axis=1)

    columns_to_average = [col for col in student_data.columns if len(student_data[col].unique()) > 1]
    if not columns_to_average:
        return None

    for column in columns_to_average:
        # Exclude 0 from the calculation
        checkTheseNulls = [0,'','0','00']
        non_zero_values = find_non_specified_uniques_from_column(student_data, column, checkTheseNulls)
        if len(non_zero_values) > 0:
           average = sum(non_zero_values) / len(non_zero_values)
           result_df[column] = average
        else:
           result_df[column] = 0

    return result_df

# Test this function below
# df = calculate_averages_for_student(sample_student)
# df_transposed = df.T
# check_these = [col for col in sample_student.columns if len(sample_student[col].unique()) > 1]
# df

Alright, not let us check to see how we can consolidate student scores. We will take a look at a few unique students who are repeated much, and take a look at the differences between their unique rows.


In [71]:
sample_student = ld_scores[ld_scores['studentId'] == 1488823]
columns_with_multiple_values = [col for col in sample_student.columns if len(sample_student[col].unique()) > 1]
for column in columns_with_multiple_values:
  print(column)
  print(sample_student[column].unique())

id
[7692595 7692587 7692593 7692592 7692591 7692590 7692589 7692588 7692594
 7692586 7692584 7692583 7692582 7004194 5426110 4713836 4707976 7692585]
measurementTypeId
[18 10 16 15 14 13 12 11 17  8  3  2  1  6  5  4  9  7]
value_starReading
[  0 519 419 546]
value_starMath
[  0 743 618 541]
value_acapELA
[  0 542 551]
value_acapMath
[  0 564 538]
value_acapMathLevel
[0 3]
value_acapELALevel
[0 3 4]
value_acapReadSubscore
[  0 552]
value_lexileStar
[  0 785 575 835]
value_acapLexile
[  0 695 465]
value_starReadingUnified
[   0 1018  980 1027  950]
value_starMathUnified
[   0 1064  987  939  949]
value_caseMathLevelExt
['' '0']
value_AcapElaRc1
[0 2]
value_AcapElaRc2
[0 3]
value_AcapElaRc3
[0 3]
value_AcapElaRc4
[0 3]
value_AcapElaRc5
[0 3]
value_AcapElaRc6
[0 3]
value_AcapElaRc7
[0 2]
value_AcapElaRc8
[0 3]
value_AcapElaRc9
[0 3]
value_AcapElaRc10
[0 2]
value_AcapMathRc1
[0 3]
value_AcapMathRc2
[0 2]
value_AcapMathRc3
[0 3]
value_AcapMathRc4
[0 3]


Great, we can use the function we just made and assume that it will only affect the test values for all students.

In [45]:
ld_scores_experiment = ld_scores.copy()
student_ids = ld_scores_experiment['studentId'].unique()
result_df = pd.DataFrame()

i=0
SIlength = int(len(student_ids))

for j in tqdm(range(i, SIlength)):
    student_id = student_ids[j]
    student_data = ld_scores[ld_scores['studentId'] == student_id]
    avg_df = calculate_averages_for_student(student_data)
    #print(avg_df)
    #avg_df['studentId'] = student_id # Add the studentId to the new row
    result_df = pd.concat([result_df, avg_df], ignore_index=True)
    # if(i%500==0):
    #   print(f"Processed {i} of {SIlength} students")
    # i = i+1

result_df

  0%|          | 0/11633 [00:00<?, ?it/s]

,id,districtId,studentId,studentLevel,year,measurementTypeId,subgroup_specialEd,subgroup_lunchStatus,subgroup_gender,subgroup_ell,...,user_Freckle Sci,user_Freckle SS,user_IXL Science,user_IXL SS,user_Freckle M Adaptive,user_Freckle M Target,user_Beable ELA,user_AR,user_Reflex M,user_Frax M
0,3268686,18,1480117,11.0,2023,4,0.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,4933508,18,1480118,12.0,2023,1,0.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,4933512,18,1480120,12.0,2023,1,0.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,7517559,18,1480121,12.0,2023,3,0.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,4933502,18,1480122,12.0,2023,1,0.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11619,7719100,18,2513584,2.0,2023,13,3.0,0,2.0,2.0,...,0,0,0,0,0,0,0,0,0,0
11620,7452821,18,2513586,0.0,2023,6,0.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
11621,7452745,18,2513587,0.0,2023,5,0.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
11622,7452823,18,2513589,0.0,2023,6,0.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


Let's take a look at this new dataset

In [46]:
result_df.head(10)

,id,districtId,studentId,studentLevel,year,measurementTypeId,subgroup_specialEd,subgroup_lunchStatus,subgroup_gender,subgroup_ell,...,user_Freckle Sci,user_Freckle SS,user_IXL Science,user_IXL SS,user_Freckle M Adaptive,user_Freckle M Target,user_Beable ELA,user_AR,user_Reflex M,user_Frax M
0,3268686,18,1480117,11.0,2023,4,0.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,4933508,18,1480118,12.0,2023,1,0.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,4933512,18,1480120,12.0,2023,1,0.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,7517559,18,1480121,12.0,2023,3,0.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,4933502,18,1480122,12.0,2023,1,0.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5,4933459,18,1480123,12.0,2023,1,0.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
6,7517562,18,1480124,12.0,2023,3,0.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
7,7513971,18,1480125,12.0,2023,7,0.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
8,4933691,18,1480126,12.0,2023,1,0.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
9,4924907,18,1480127,12.0,2023,1,0.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
result_df.shape

(11624, 726)

In [48]:
pruned_df = result_df.copy()

In [ ]:
# ld_scoresPrime[ld_scoresPrime['studentId']==1488823]['user_IXL Reading']

,user_IXL Reading
7238,0


In [49]:
ld_vendorUsage[ld_vendorUsage['studentId']==1488823]

,districtId,studentId,year,vendorId,active,usageTypeId,weeklyUsageMinutes
42138,18,1488823,2023,11333,0,6,0
42139,18,1488823,2023,72609,0,2,0
42140,18,1488823,2023,140472,0,37,120
42141,18,1488823,2023,140574,0,40,0


In [50]:
ld_trimmedVendorTypes

,usageTypeId,name,utilBucketType,id,product_name,productUse_int
0,1,IXL Reading Non,nonUser,4,IXL Reading,0
1,2,IXL Reading Partial,partialUser,5,IXL Reading,1
2,3,IXL Reading User,fullUser,6,IXL Reading,2
3,4,IXL Math Non User,nonUser,7,IXL Math,0
4,5,IXL Math Partial,partialUser,8,IXL Math,1
5,6,IXL Math User,fullUser,9,IXL Math,2
6,7,MyOn R User,fullUser,853,MyOn R,2
7,8,MyOn R Partial User,partialUser,854,MyOn R,1
8,9,MyOn R Non User,nonUser,855,MyOn R,0
9,10,Freckle R User,fullUser,1130,Freckle R,2


In [54]:
def translateMeasureIdToTarget(row, reference, target):
  #print(row)
  studentId = row['studentId']
  usageTypeId = row['usageTypeId']
  productItem = ld_trimmedVendorTypes[ld_trimmedVendorTypes['usageTypeId'] == usageTypeId]
  productName = productItem['product_name'].iloc[0]
  productUse_int = productItem['productUse_int'].iloc[0]
  rowToPopulate = -1
  try:
    columnToPopulate = "user_"+productName
    rowToPopulate = target[target['studentId'] == studentId].index[0]
  except IndexError:
    print(f"Student ID {studentId} not found in target.")

  if rowToPopulate == -1:
    return
  target.loc[rowToPopulate, columnToPopulate] = productUse_int

  #print('Student ID: ' + str(studentId) + ' Product: ' + productName + ' ProductUse_int: ' + str(productUse_int))
  return


In [55]:
for currRow in range(0, ld_trimmedVendorUsage.shape[0]):
  translateMeasureIdToTarget(row=ld_trimmedVendorUsage.loc[currRow], reference=ld_trimmedVendorTypes, target=pruned_df)


Student ID 1480143 not found in target.
Student ID 1480143 not found in target.
Student ID 1480143 not found in target.
Student ID 1480148 not found in target.
Student ID 1480148 not found in target.
Student ID 1480148 not found in target.
Student ID 1480157 not found in target.
Student ID 1480157 not found in target.
Student ID 1480157 not found in target.
Student ID 1480185 not found in target.
Student ID 1480185 not found in target.
Student ID 1480185 not found in target.
Student ID 1480185 not found in target.
Student ID 1480185 not found in target.
Student ID 1480196 not found in target.
Student ID 1480196 not found in target.
Student ID 1480196 not found in target.
Student ID 1480220 not found in target.
Student ID 1480220 not found in target.
Student ID 1480220 not found in target.
Student ID 1480225 not found in target.
Student ID 1480225 not found in target.
Student ID 1480225 not found in target.
Student ID 1480225 not found in target.
Student ID 1480225 not found in target.


In [58]:
#so how many students did we not cover?
primeVendorUsage = ld_trimmedVendorUsage.drop_duplicates(subset=['studentId'], keep='first')
primeVendorUsage = primeVendorUsage.reset_index(drop=True)
scores_missing = 0

for currRow in range(0, primeVendorUsage.shape[0]):
  if pruned_df[pruned_df['studentId']==(primeVendorUsage.loc[currRow]['studentId'])].empty:
    scores_missing += 1
print("Students missing in scores data (ideally is 0): " + str(scores_missing))

products_missing = 0

for currRow in range(0, pruned_df.shape[0]):
  if primeVendorUsage[primeVendorUsage['studentId']==(pruned_df.loc[currRow]['studentId'])].empty:
    products_missing += 1
print("Students without product data (not unexpected to be nonzero): " + str(products_missing))

Students missing in scores data (ideally is 0): 692
Students without product data (not unexpected to be nonzero): 662


In [59]:
# prompt: make a dataframe with one column: studentId, populated with all the unique studentIds from primeVendorUsage. Add one more column called 'existsInScores'.

# Create an empty DataFrame with 'studentId' and 'existsInScores' columns
student_existence_df = pd.DataFrame(columns=['studentId', 'existsInScores'])

# Iterate through unique student IDs in primeVendorUsage
for student_id in primeVendorUsage['studentId'].unique():
  # Check if the student ID exists in ld_scoresPrime
  exists_in_scores = not pruned_df[pruned_df['studentId'] == student_id].empty

  # Append a new row to the DataFrame with the student ID and existence status
  student_existence_df = pd.concat([student_existence_df, pd.DataFrame({'studentId': [student_id], 'existsInScores': [exists_in_scores]})], ignore_index=True)

# Print the resulting DataFrame
print(student_existence_df)


NameError: name 'ld_scoresPrime' is not defined

In [60]:
student_existence_df['existsInScores'].value_counts()

,count
existsInScores,


In [61]:
student_existence_df.to_csv('/content/drive/MyDrive/Level Data Datasets/vendorUsagetoScoresExistence.csv')


In [62]:
students_sans_scores_df = student_existence_df[student_existence_df['existsInScores']==False]
students_sans_scores_df.to_csv('/content/drive/MyDrive/Level Data Datasets/vendorStudentsSansScores.csv')


In [64]:
user_cols = pruned_df.filter(regex='^user_').columns
for column in user_cols:
  print(pruned_df[column].value_counts())


user_IXL Reading
2    5515
0    3794
1    2315
Name: count, dtype: int64
user_IXL Math
2    5417
0    3888
1    2319
Name: count, dtype: int64
user_MyOn R
0    11624
Name: count, dtype: int64
user_Freckle R
0    11624
Name: count, dtype: int64
user_Freckle M
0    11624
Name: count, dtype: int64
user_Freckle Sci
0    11624
Name: count, dtype: int64
user_Freckle SS
0    11624
Name: count, dtype: int64
user_IXL Science
0    10822
1      602
2      200
Name: count, dtype: int64
user_IXL SS
0    11252
1      259
2      113
Name: count, dtype: int64
user_Freckle M Adaptive
0    11624
Name: count, dtype: int64
user_Freckle M Target
0    11624
Name: count, dtype: int64
user_Beable ELA
0    11624
Name: count, dtype: int64
user_AR
0    8996
2    1683
1     945
Name: count, dtype: int64
user_Reflex M
0    10459
1     1004
2      161
Name: count, dtype: int64
user_Frax M
0    11624
Name: count, dtype: int64


It looks like we can eliminate the following products, since no students in the districts use them:
 - Frax M
 - Freckle M
 - Freckle M Target
 - Freckle M Adaptive
 - Beable ELA
 - Freckle R
 - Freckle Sci
 - Freckle SS
 - MyOn R

 So, the vendor products we **can** start to look at follow:
 - Reflex M
 - AR
 - IXL SS
 - IXL Science
 - IXL Math
 - IXL Reading

In [65]:
user_cols_drop = ['user_MyOn R', 'user_Freckle R', 'user_Freckle M', 'user_Freckle Sci', 'user_Freckle SS', 'user_Freckle M Adaptive', 'user_Freckle M Target', 'user_Beable ELA', 'user_Frax M']
pruned_df = pruned_df.drop(columns=user_cols_drop)

In [67]:
pruned_df['user_IXL SS'].value_counts()

,count
user_IXL SS,
0,11252
1,259
2,113


In [68]:
pruned_df.head()

,id,districtId,studentId,studentLevel,year,measurementTypeId,subgroup_specialEd,subgroup_lunchStatus,subgroup_gender,subgroup_ell,...,value_AcapSciRc4,value_fsaWriting,value_fsaWritingLevel,value_acadienceReadingCompositeLevel,user_IXL Reading,user_IXL Math,user_IXL Science,user_IXL SS,user_AR,user_Reflex M
0,3268686,18,1480117,11.0,2023,4,0.0,0,0.0,0.0,...,0.0,0,0,0,2,2,0,0,0,0
1,4933508,18,1480118,12.0,2023,1,0.0,0,0.0,0.0,...,0.0,0,0,0,2,2,0,0,0,0
2,4933512,18,1480120,12.0,2023,1,0.0,0,0.0,0.0,...,0.0,0,0,0,2,2,0,0,0,0
3,7517559,18,1480121,12.0,2023,3,0.0,0,0.0,0.0,...,0.0,0,0,0,2,2,0,0,0,0
4,4933502,18,1480122,12.0,2023,1,0.0,0,0.0,0.0,...,0.0,0,0,0,2,2,0,0,0,0


In [69]:
pruned_df.to_csv(r'/content/drive/MyDrive/Level Data Datasets/District_18_pruned_score_averages.csv', index=False)


In [70]:
verify_averages = pd.read_csv(r'/content/drive/MyDrive/Level Data Datasets/District_18_pruned_score_averages.csv')
verify_averages.head()

,id,districtId,studentId,studentLevel,year,measurementTypeId,subgroup_specialEd,subgroup_lunchStatus,subgroup_gender,subgroup_ell,...,value_AcapSciRc4,value_fsaWriting,value_fsaWritingLevel,value_acadienceReadingCompositeLevel,user_IXL Reading,user_IXL Math,user_IXL Science,user_IXL SS,user_AR,user_Reflex M
0,3268686,18,1480117,11.0,2023,4,0.0,0,0.0,0.0,...,0.0,0,0,0,2,2,0,0,0,0
1,4933508,18,1480118,12.0,2023,1,0.0,0,0.0,0.0,...,0.0,0,0,0,2,2,0,0,0,0
2,4933512,18,1480120,12.0,2023,1,0.0,0,0.0,0.0,...,0.0,0,0,0,2,2,0,0,0,0
3,7517559,18,1480121,12.0,2023,3,0.0,0,0.0,0.0,...,0.0,0,0,0,2,2,0,0,0,0
4,4933502,18,1480122,12.0,2023,1,0.0,0,0.0,0.0,...,0.0,0,0,0,2,2,0,0,0,0
